# KBReport에서 데이터 가져오기
http://www.kbreport.com/teams/main

* http://www.kbreport.com/teams/main 
    * ##팀기록 메인화면
* ?rows=20&order=TPCT&orderType=DESC    
* &teamId=8                             
    * ##한화의 팀 아이디=8
* &defense_no=                          
    * ##포지션 선택
* &year_from=2012                       
    * ##시작년도
* &year_to=2016                         
    * ##종료년도
* &split01=day                          
    * ##첫번째 분류
* &split02_1=2012-03-01                 
    * ##두번째 분류(날짜선택)
* &split02_2=2012-03-01

In [1]:
#-*- coding: utf-8 -*-
#import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import json
import numpy as np
import pickle
import copy

In [2]:
f = open('Data_HanHwa_GameDate_from_KBO.txt', 'rb')

DateList = pickle.load(f)

f.close()

In [3]:
DateList12_4 = DateList[266:283]
DateList12_5 = DateList[283:310]
DateList12_6 = DateList[310:334]
DateList12_7 = DateList[334:352]
DateList12_8 = DateList[352:373]
DateList12_9 = DateList[373:395]
DateList12_10 = DateList[395:398]

In [9]:
print DateList12_5

['2012-05-01', '2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06', '2012-05-08', '2012-05-09', '2012-05-10', '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20', '2012-05-22', '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-29', '2012-05-30', '2012-05-31']


In [10]:
## 필요 리스트 ##

##YearList = ['2012', '2013', '2014', '2015', '2016']
##MonthList= ['03', '04', '05', '06', '07', '08', '09', '10']
##Days = 31
##Team = 'HH'
sleeptime = 5
retry = 5

##테스트용
YearList = ['2012']
MonthList= ['04']
Days = 31

In [ ]:
## 데이터 파싱 ##

driver = webdriver.Chrome("C:\Users\Kim Jin Hak\Desktop\chromedriver.exe")
SiteURL = 'http://www.kbreport.com/teams/main?rows=20&order=TPCT&orderType=DESC&teamId=8&defense_no=&year_from=2012&year_to=2016&split01=day&split02_1=' 

i = 0
result = []
for date in DateList12_5:
    url = SiteURL + date + '&split02_2=' + date
            
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    time.sleep(sleeptime)
            
    for re1 in range(retry):
        try:
#            print "%s-%s-%d 당시 table 찾기 시도" % (year, month, day+1)
            table = soup.find('table', attrs={'class':'ltb-table responsive'})
        except NameError as eN:
#            print eN, "%d회 시도: table 찾기 실패" % (re1+1)
            time.sleep(sleeptime)
            pass
        else:
#            print "table 찾기 성공"
            for re2 in range(retry): 
                try:
#                    print "%s-%s-%d 당시 tbody 찾기 시도" % (year, month, day+1)
                    table_body = table.find('tbody')
                except AttributeError as eA:
#                    print eA, "%d회 시도: tbody 찾기 실패" % (re2+1)
                    time.sleep(sleeptime)
                    pass
                else:
#                    print "tbody 찾기 성공"
                    rows = table_body.find_all('tr')
                    try:
                        print "%s 당시 tr 찾기 시도" % (date)
                        row = rows[1].find_all(text=True)
                    except IndexError as eI:
                        print eI, "tr 찾기 실패"
                        result.append([])
                        result[i].append(date)
                        i = i+1
                        pass
                    else:
                        print "tr 찾기 성공"
                        result.append([])
                        result[i] = row
                        result[i][0] = date
                        i = i+1
                    break
            break
            
driver.close()
driver.quit()

print "finish"

2012-05-01 당시 tr 찾기 시도
tr 찾기 성공
2012-05-02 당시 tr 찾기 시도
tr 찾기 성공


In [ ]:
## KBReport에서 제대로 파싱이 되었는지 확인 ##

print len(DateList12_5)
print len(result)

resultdate = []
for i in result:
    resultdate.append(i[0])
    
for i in DateList12_5:
    if i not in resultdate:
        print i

In [92]:
filename = 'Parsing_TeamData_from_KBReport' + '12_05' + '.txt'
f = open(filename, 'wb')

pickle.dump(result, f)

f.close()